##1 Install required libraries

Install all the required libraries and helper functions

In [ ]:
%pip install chromadb --quiet
%pip install sentence_transformers --quiet
%pip install pypdf --quiet
%pip install langchain --quiet
%pip install tqdm --quiet

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter


from pypdf import PdfReader

from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient
from chromadb.utils import embedding_functions

import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.0

## 2 Initialize a Persistent ChromaDB client with a proper Google Drive connection

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
# Initialize ChromaDB client with Google Drive connection
chromaDB_path = '/content/drive/MyDrive/Colab Notebooks/ChromaDBData'


In [ ]:
# Check if the chromadb_path exists or not. If so, delete all the files and folders in chromadb_path. But before deleting get the permission from the user.

import os
import shutil

def delete_all_files_and_folders(chromaDB_path):
  if os.path.exists(chromaDB_path):
    print(f"The directory '{chromaDB_path}' already exists.")
    permission = input("Do you want to delete all the files and folders in this directory? (y/n): ")
    if permission == "y":
      shutil.rmtree(chromaDB_path)
      print(f"All files and folders in '{chromaDB_path}' have been deleted.")
    else:
      print("No action taken.")
  else:
    print(f"The directory '{chromaDB_path}' does not exist.")



In [ ]:
delete_all_files_and_folders(chromaDB_path)

The directory '/content/drive/MyDrive/Colab Notebooks/ChromaDBData' already exists.
Do you want to delete all the files and folders in this directory? (y/n): y
All files and folders in '/content/drive/MyDrive/Colab Notebooks/ChromaDBData' have been deleted.


## 3 Define PersistentClient

Let's re-define the **create_chroma_client** function from the previous part so that this time we initialize a **persistent** ChromaDB client:

In [ ]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient


In [ ]:
def create_chroma_client(collection_name, embedding_function, chromaDB_path ):
  if chromaDB_path is not None:
    chroma_client = PersistentClient(path=chromaDB_path,
                                     settings=Settings(),
                                     tenant=DEFAULT_TENANT,
                                     database=DEFAULT_DATABASE,)
  else:
    chroma_client = Client()

  chroma_collection = chroma_client.get_or_create_collection(
      collection_name,
      embedding_function=embedding_function)

  return chroma_client, chroma_collection

## 4 Create a collection as usual

In [ ]:
collection_name = "Papers"
sentence_transformer_model="distiluse-base-multilingual-cased-v1"
embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
chroma_client, chroma_collection = create_chroma_client(collection_name,
                                                        embedding_function,
                                                        chromaDB_path)

### Check the created collection:

In [ ]:
# Verify collection properties
print(f"Collection name: {chroma_collection.name}")
print(f"Number of documents in collection: {chroma_collection.count()}")

# List all collections in the client
print("All collections in ChromaDB client:")
for collection in chroma_client.list_collections():
    print(collection.name)

Collection name: Papers
Number of documents in collection: 0
All collections in ChromaDB client:
Papers


## 5 Define helper functions

In [ ]:
from google.colab import files
def upload_multiple_files():
  uploaded = files.upload()
  file_names = list()
  for fn in uploaded.keys():
    #print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    file_names.append(fn)
  return file_names

In [ ]:
def convert_PDF_Text(pdf_path):
  reader = PdfReader(pdf_path)
  pdf_texts = [p.extract_text().strip() for p in reader.pages]
  # Filter the empty strings
  pdf_texts = [text for text in pdf_texts if text]
  print("Document: ",pdf_path," chunk size: ", len(pdf_texts))
  return pdf_texts

In [ ]:
def convert_Page_ChunkinChar(pdf_texts, chunk_size = 1500, chunk_overlap=0 ):
  character_splitter = RecursiveCharacterTextSplitter(
      separators=["\n\n", "\n", ". ", " ", ""],
      chunk_size=1500,
      chunk_overlap=0
)
  character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))
  print(f"\nTotal number of chunks (document splited by max char = 1500): \
        {len(character_split_texts)}")
  return character_split_texts

In [ ]:
def convert_Chunk_Token(text_chunksinChar,sentence_transformer_model, chunk_overlap=0,tokens_per_chunk=128 ):
  token_splitter = SentenceTransformersTokenTextSplitter(
      chunk_overlap=0,
      model_name=sentence_transformer_model,
      tokens_per_chunk=128)

  text_chunksinTokens = []
  for text in text_chunksinChar:
      text_chunksinTokens += token_splitter.split_text(text)
  print(f"\nTotal number of chunks (document splited by 128 tokens per chunk):\
       {len(text_chunksinTokens)}")
  return text_chunksinTokens

In [ ]:
def add_meta_data(text_chunksinTokens, title, category, initial_id):
  ids = [str(i+initial_id) for i in range(len(text_chunksinTokens))]
  metadata = {
      'document': title,
      'category': category
  }
  metadatas = [ metadata for i in range(len(text_chunksinTokens))]
  return ids, metadatas

In [ ]:
def add_document_to_collection(ids, metadatas, text_chunksinTokens, chroma_collection):
  print("Before inserting, the size of the collection: ", chroma_collection.count())
  chroma_collection.add(ids=ids, metadatas= metadatas, documents=text_chunksinTokens)
  print("After inserting, the size of the collection: ", chroma_collection.count())
  return chroma_collection

In [ ]:
def retrieveDocs(chroma_collection, query, n_results=5, return_only_docs=False):
    results = chroma_collection.query(query_texts=[query],
                                      include= [ "documents","metadatas",'distances' ],
                                      n_results=n_results)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results

In [ ]:
def show_results(results, return_only_docs=False):

  if return_only_docs:
    retrieved_documents = results
    if len(retrieved_documents) == 0:
      print("No results found.")
      return
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print("\tDocument Text: ")
      display(to_markdown(doc));
  else:

      retrieved_documents = results['documents'][0]
      if len(retrieved_documents) == 0:
          print("No results found.")
          return
      retrieved_documents_metadata = results['metadatas'][0]
      retrieved_documents_distances = results['distances'][0]
      print("------- retreived documents -------\n")

      for i, doc in enumerate(retrieved_documents):
          print(f"Document {i+1}:")
          print("\tDocument Text: ")
          display(to_markdown(doc));
          print(f"\tDocument Source: {retrieved_documents_metadata[i]['document']}")
          print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
          print(f"\tDocument Distance: {retrieved_documents_distances[i]}")


In [ ]:
def load_multiple_pdfs_to_ChromaDB(collection_name,sentence_transformer_model,
                                   chromaDB_path):

  collection_name= collection_name
  category= "Journal Paper"
  sentence_transformer_model=sentence_transformer_model
  embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)
  chroma_client, chroma_collection = create_chroma_client(collection_name, embedding_function, chromaDB_path)
  current_id = chroma_collection.count()
  file_names = upload_multiple_files()
  for file_name in file_names:
    print(f"Document: {file_name} is being processed to be added to the {chroma_collection.name} {chroma_collection.count()}")
    print(f"current_id: {current_id} ")
    pdf_texts = convert_PDF_Text(file_name)
    text_chunksinChar = convert_Page_ChunkinChar(pdf_texts)
    text_chunksinTokens = convert_Chunk_Token(text_chunksinChar,sentence_transformer_model)
    ids,metadatas = add_meta_data(text_chunksinTokens,file_name,category, current_id)
    current_id = current_id + len(text_chunksinTokens)
    chroma_collection = add_document_to_collection(ids, metadatas, text_chunksinTokens, chroma_collection)
    print(f"Document: {file_name} added to the collection: {chroma_collection.count()}")
  return  chroma_client, chroma_collection

## 6 Upload Multiple Documents and Create Knowledge Base

Run load_multiple_pdfs_to_ChromaDB() to fill in the colection

In [ ]:
chroma_client, chroma_collection= load_multiple_pdfs_to_ChromaDB(collection_name,sentence_transformer_model, chromaDB_path)

Saving 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf to 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
Saving 24 Attention Is All You Need.pdf to 24 Attention Is All You Need.pdf
Saving 29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf to 29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf
Saving 30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf to 30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf
Saving 73 Analyzing Students Academic Success.pdf to 73 Analyzing Students Academic Success.pdf
Saving 77 Arac Park Yerlerinin Doluluk.pdf to 77 Arac Park Yerlerinin Doluluk.pdf
Document: 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf is being processed to be added to the Papers 0
current_id: 0 
Document:  15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf  chunk size:  1

Total number of chunks (document splited by 

## 7 Test the Knowledge Base

Query the Knowledge Base using the persistent ChromaDB client and & collection

In [ ]:
query = "What are the main difference in active and passive path scheduling?"

'''
In 16 A Local Optimization Technique for Assigning New Targets ABSTRACT:

Route planning can be static or dynamic. In static route planning, routes are
constructed according to given UAVs and targets and do not change during
the mission. However, in dynamic route planning, number of routes or UAVs
can alter which requires the update of existing routes to adopt these changes.

'''

'\nIn 16 A Local Optimization Technique for Assigning New Targets ABSTRACT:\n\nRoute planning can be static or dynamic. In static route planning, routes are\nconstructed according to given UAVs and targets and do not change during\nthe mission. However, in dynamic route planning, number of routes or UAVs\ncan alter which requires the update of existing routes to adopt these changes.\n\n'

In [ ]:
retrieved_documents=retrieveDocs(chroma_collection, query, 10)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> memory gets full and overflow s. For a successful schedule, the crucial information is the travelling time between sensor nodes in the field. In most cases, it is assumed that the travelling time is known a priori and remains the same all the time. However, in reality, due to various reasons, travelling times can change in course of time and, hence, the planned schedule may not produce the desired output. In this study, we propose an improved scheduling method consid ering uncertainty in travelling time. Simulation experiments justify the expected success of the proposed method. Index Terms [UNK] Wireless Sensor Networks

	Document Source: 29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.3394520282745361
Document 2:
	Document Text: 


> ##lected data for a given period of time, namely tour time. Provided below are the details of the MES / TSACO implementation 3. 2 The Time - Sensitive ACO A combinatorial optimization problem can be static or dynamic with respect to the given characteristics of the problem. In static problems, the underly - ing system properties stay the same throughout the problem - solution pro - cess. A typical example of these kinds of problems is the Traveling Sales - man Problem ( TSP ). In the TSP, there are a number of towns ( vertices ) con - nected to each

	Document Source: 30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.3876214027404785
Document 3:
	Document Text: 


...

	Document Source: 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.4616098403930664
Document 9:
	Document Text: 


> Second, the considered chainedcourses are limited to three mathematical courses ( Calculus I, Calculus II, and [UNK] Equa - tions ). Third, among the several curriculum devel - opment methods, only two ( Spiral and Linear ) areselected for this research. Lastly, since there are [UNK] records stating which courses were designed according to which curriculum development meth - odology, the [UNK] of the chains are doneaccording to the [UNK] of the cirriculum devel - opment methods. Murat Karakaya et al. 368

	Document Source: 73 Analyzing Students Academic Success.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.4657018184661865
Document 10:
	Document Text: 


> Table 1 : Maximum path lengths, per - layer complexity and minimum number of sequential operations for different layer types. nis the sequence length, dis the representation dimension, kis the kernel size of convolutions and rthe size of the neighborhood in restricted self - attention. Layer Type Complexity per Layer Sequential Maximum Path Length Operations Self - Attention O ( n2 · d ) O ( 1 ) O ( 1 ) Recurrent O ( n · d2 ) O ( n ) O ( n ) Convolutional O ( k · n · d2 ) O ( 1 ) O

	Document Source: 24 Attention Is All You Need.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.4922304153442383


## 8 Observe the ChromeDB saved to the provided path

List the folders and files in the chromaDB_path

In [ ]:
!ls "{chromaDB_path}"

chroma.sqlite3	d737a052-8af6-4c2c-8cb6-1147eb4b1b9c


## YES WE DID IT!

# LOAD A KNOWLEDGE BASE FROM A PERSISTENT CHROMADB



Let's kill the kernel so we ensure that nothing remains in the memory from all the above ChromaDB instance.

In [ ]:
from google.colab import runtime
# Disconnect from the runtime
#!kill -9 -1

##1 Connect to source directory

First get connected to the ChromaDB directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# change directory to chromaDB folder
chromaDB_path = '/content/drive/MyDrive/Colab Notebooks/ChromaDBData'
%cd {chromaDB_path}


/content/drive/MyDrive/Colab Notebooks/ChromaDBData


### Check that if chromadb_path exist or not and if exists does it contain chromadb files and folders

In [ ]:
import os
if os.path.exists(chromaDB_path):
    print(f"The directory '{chromaDB_path}' exists.")

    # Check if the directory contains ChromaDB files and folders
    chromadb_files_and_folders = os.listdir(chromaDB_path)
    if any(file_or_folder.startswith('chroma') for file_or_folder in chromadb_files_and_folders):
        print("The directory contains ChromaDB files and folders.")
    else:
        print("The directory does not contain ChromaDB files and folders.")
else:
    print(f"The directory '{chromadb_path}' does not exist.")


The directory '/content/drive/MyDrive/Colab Notebooks/ChromaDBData' exists.
The directory contains ChromaDB files and folders.


##2 Install required libraries

Secondly install all the required libraries and helper functions

In [ ]:
%pip install chromadb --quiet
%pip install sentence_transformers --quiet

In [ ]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient
from chromadb.utils import embedding_functions


In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def retrieveDocs(chroma_collection, query, n_results=5,
                 return_only_docs=False, filterType=None, filterValue=None):
    if filterType is not None and filterValue is not None:
        results = chroma_collection.query(
            query_texts=[query],
            include=["documents", "metadatas", "distances"],
            where={filterType: filterValue},
            n_results=n_results)

    else:
        results = chroma_collection.query(
            query_texts=[query],
            include= [ "documents","metadatas",'distances' ],
            n_results=n_results)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results

In [ ]:
def show_results(results, return_only_docs=False):

  if return_only_docs:
    retrieved_documents = results
    if len(retrieved_documents) == 0:
      print("No results found.")
      return
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print("\tDocument Text: ")
      display(to_markdown(doc));
  else:

      retrieved_documents = results['documents'][0]
      if len(retrieved_documents) == 0:
          print("No results found.")
          return
      retrieved_documents_metadata = results['metadatas'][0]
      retrieved_documents_distances = results['distances'][0]
      print("------- retreived documents -------\n")

      for i, doc in enumerate(retrieved_documents):
          print(f"Document {i+1}:")
          print("\tDocument Text: ")
          display(to_markdown(doc));
          print(f"\tDocument Source: {retrieved_documents_metadata[i]['document']}")
          print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
          print(f"\tDocument Distance: {retrieved_documents_distances[i]}")


##3 Initailizing

 Now, we can begin to upload the persistent ChromaDB from the location by initailizing
*  the chromaDB client
*  the chromaDB collections

In [ ]:
chroma_client = PersistentClient(path=chromaDB_path,
                                     settings=Settings(),
                                     tenant=DEFAULT_TENANT,
                                     database=DEFAULT_DATABASE,)

In [ ]:
chroma_client.list_collections()

[Collection(id=6e80f120-e77b-4599-ab9a-24a64407756b, name=Papers)]

In [ ]:
collection_name = "Papers"
sentence_transformer_model="distiluse-base-multilingual-cased-v1"
embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model)


In [ ]:
chroma_collection = chroma_client.get_or_create_collection(
      collection_name,
      embedding_function=embedding_function)

In [ ]:
# Verify collection properties
print(f"Collection name: {chroma_collection.name}")  # Access the name attribute directly
print(f"Number of documents in collection: {chroma_collection.count()}")

# List all collections in the client
print("All collections in ChromaDB client:")
for collection in chroma_client.list_collections():
    print(collection.name)

Collection name: Papers
Number of documents in collection: 331
All collections in ChromaDB client:
Papers


##4 Test

Test the loaded ChromeDB client and the collection

In [ ]:
chroma_collection.get(['0'])

{'ids': ['0'],
 'embeddings': None,
 'metadatas': [{'category': 'Journal Paper',
   'document': '15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf'}],
 'documents': ['UAV Route Planning For Maximum Target Coverage Murat Karakaya Department of Computer Engineering, Atilim University Incek / Ankara TURKEY kmkarakaya @ atilim. edu. tr Abstract The importance and the impact of using Unmanned Aerial Vehicles ( UAVs ) in military and civil operations are increasing. One of the challenges in effectively tasking these expensive vehicles is planning the flight routes to monitor the targets. This problem is related with the Multiple Travelling Salesman Problem ( mTSP ) and the Vehicle Routing Problem ( VRP ). In these well - defined problem'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [ ]:
query = "What is Target Coverage?"

In [ ]:
retrieved_documents=retrieveDocs(chroma_collection, query, 10)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> s, it is mostly assumed that travelling salesm en or vehicles should visit all the targets and the target function is defined as to find a minimum - distan t route. Even, i n the constraint versions of the mTSP and VRP, some other restrictions ( visiting time windows, number of depots, etc. ) are included ; it is still assumed that there exists enough number of travelling salesm en or vehicle s to cover all the give n locations. However, in reality the number and flight range of UAVs might be insufficient to cover all the targets. As a result, t he maxim

	Document Source: 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.2466243505477905
Document 2:
	Document Text: 


> impact of cirriculumdeveloment method. As seen above references, there have been a considerable interest on the pre - requisite coursesin the literature. However, the scope, goals, and results of the above - mentioned studies are not completely in line with those of the present work. One of the studies with similar scope and goals wasconducted by Anderson et al. [ 25 ]. They found thatsuccessfully completing calculus and economicscourses positively [UNK] the achieved success insubsequent economics courses. In another work, McMillan and Adeyemi focused on the success relationships between the

	Document Source: 73 Analyzing Students Academic Success.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.3942387104034424
Document 3:
	Document Text: 


...

	Document Source: 73 Analyzing Students Academic Success.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.5079680681228638
Document 9:
	Document Text: 


> requisite chains periodi - cally with empirical data so that pre - requisites and their follow - up courses should be well designed to support student success throughoutthe curriculum. 14. Conclusions The goal of this study is two - fold. The [UNK] goal is to examine the impact of the pre - requisite courses onthe success in a follow - up course for two mathe - matics course chains. The second one is to investi - gate the impact of the selected cirriculumdevelopment method on the success relationshipsbetween a pre - requisite course and follow

	Document Source: 73 Analyzing Students Academic Success.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.5144762992858887
Document 10:
	Document Text: 


> lliklerini ezberleyeceği ( overfitting ) için başarı oranı gerçeği yansıtmayacak kadar yüksek çıkabilecektir. B irbirinden farklı görüntüler den oluşan ü ç veri setinin kullanımı ile D ESA [UNK] nın sınıflandırma başarı m ı hakkında daha gerçekçi sonuçlar elde edilmek i stenmiştir. Şekil 2 : Veri s etlerinde b ulunan p ark y eri g örüntülerinden ö rnekler B. Derin Evrişimsel Sinir Ağları Bu çalışma

	Document Source: 77 Arac Park Yerlerinin Doluluk.pdf
	Document Source Type: Journal Paper
	Document Distance: 1.5149482488632202


## YES! WE DID IT!

# CONNECT TO AN LLM: GOOGLE GEMINI

## 1 Install & Import Libraries

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
import os
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

## 2 Define Helper Functions

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def build_chatBot(system_instruction):
  model = genai.GenerativeModel('gemini-1.5-flash-latest', system_instruction=system_instruction)
  chat = model.start_chat(history=[])
  return chat

In [ ]:
def generate_LLM_answer(prompt, context, chat):
  response = chat.send_message( prompt + context)
  return response.text

## 3 Connect to the LLM via the Chat API

In [ ]:
# Used to securely store your API key
from google.colab import userdata
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
system_prompt= """ You are an attentive and supportive academic assistant.
Your role is to provide assistance based solely on the provided context.

Here’s how we’ll proceed:
1. I will provide you with a question and related text excerpt.
2. Your task is to answer the question using only the provided partial texts.
3. If the answer isn’t explicitly found within the given context,
respond with 'I don't know'.
4. After each response, please provide a detailed explanation.
Break down your answer step by step and relate it directly to the provided context.
5. Sometimes, I will ask questions about the chat session, such as summarize
the chat or list the question etc. For this kind of questions do not try
to use the provided partial texts.
6. Generate the answer in the same language of the given question.

If you're ready, I'll provide you with the question and the context.
"""

In [ ]:
RAG_LLM = build_chatBot(system_prompt)

## 4 Test the LLM connection

In [ ]:
prompt="What is FC?"
context= """FC lets developers create a description
of a F in their code, then pass that description to a language
model in a request.

The response from the model includes the name of
a F that matches the description and the arguments to call it with.
FC lets you use F as tools in generative AI applications,
and you can define more than one F within a single request.
"""

In [ ]:
response=generate_LLM_answer(prompt, context,RAG_LLM)
to_markdown(response)

> I don't know.
> 
> The provided text describes the functionality of FC and mentions that it lets developers create descriptions of "F" in their code. However, it does not explicitly define what "FC" or "F" actually are. The text focuses on the purpose and usage of FC, but it doesn't provide a clear definition of these terms. 


In [ ]:
RAG_LLM.history

[parts {
   text: "What is FC?FC lets developers create a description\nof a F in their code, then pass that description to a language\nmodel in a request.\n\nThe response from the model includes the name of\na F that matches the description and the arguments to call it with.\nFC lets you use F as tools in generative AI applications,\nand you can define more than one F within a single request.\n"
 }
 role: "user",
 parts {
   text: "I don\'t know.\n\nThe provided text describes the functionality of FC and mentions that it lets developers create descriptions of \"F\" in their code. However, it does not explicitly define what \"FC\" or \"F\" actually are. The text focuses on the purpose and usage of FC, but it doesn\'t provide a clear definition of these terms. \n"
 }
 role: "model"]

In [ ]:
RAG_LLM.history.clear()
RAG_LLM.history

[]

# CREATE THE RAG PIPELINE FOR THE EXISTING KNOWLEDGE BASE

## 1 A simple RAG Pipeline

* preparea summary for the Knowledge Base
* get the query from the user
* query the Knowledge Base
* get the related chunks from the Knowledge Base
* combine the query + context from Knowledge Base
* submit the prompt (query + context) to the LLM
* get the response from the LLM

In [ ]:
# Verify collection properties
print(f"Collection name: {chroma_collection.name}")  # Access the name attribute directly
print(f"Number of documents in collection: {chroma_collection.count()}")

# List all collections in the client
print("All collections in ChromaDB client:")
for collection in chroma_client.list_collections():
    print(collection.name)

Collection name: Papers
Number of documents in collection: 331
All collections in ChromaDB client:
Papers


In [ ]:
def summarize_collection(chroma_collection):
  summary = [] # Initialize summary as a list
  print("Summarizing the collection...")
  # Verify collection properties
  print(f"\t Collection name: {chroma_collection.name}")  # Access the name attribute directly
  print(f"\t Number of document chunks in collection: {chroma_collection.count()}")
  summary.append(f"Collection name: {chroma_collection.name}") # Append to the list
  summary.append(f"Number of document chunks in collection: {chroma_collection.count()}")
  # Print distinct metadata "document" for each chunk in the collection
  print("\t Distinct 'document' metadata in the collection:")
  distinct_documents = set()  # Use a set to store unique document names

  # Iterate over chunks in the collection
  for chunk_id in range(chroma_collection.count()):
      metadata = chroma_collection.get([str(chunk_id)])['metadatas'][0]  # Get metadata for the chunk
      document_name = metadata.get("document", "Unknown")  # Get document metadata; default to "Unknown" if not present
      distinct_documents.add(document_name)  # Add document name to set for uniqueness

  # Print all distinct document names
  summary.append("Documents:")
  for document_name in distinct_documents:
      print("\t ",document_name)
      summary.append(document_name) # Append to the list

  print("Collection summarization completed.")

  # Join the list elements into a single string
  summary_string = "\n ".join(summary)
  return summary_string

In [ ]:
s=summarize_collection(chroma_collection)

Summarizing the collection...
	 Collection name: Papers
	 Number of document chunks in collection: 331
	 Distinct 'document' metadata in the collection:
	  30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf
	  77 Arac Park Yerlerinin Doluluk.pdf
	  15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
	  24 Attention Is All You Need.pdf
	  29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf
	  73 Analyzing Students Academic Success.pdf
Collection summarization completed.


In [ ]:
print(s)

Collection name: Papers
 Number of document chunks in collection: 331
 Documents:
 30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf
 77 Arac Park Yerlerinin Doluluk.pdf
 15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
 24 Attention Is All You Need.pdf
 29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf
 73 Analyzing Students Academic Success.pdf


In [ ]:
def generateAnswer(RAG_LLM, chroma_collection,query,n_results=5, only_response=True):
    retrieved_documents= retrieveDocs(chroma_collection, query, 10, return_only_docs=True)
    prompt = "QUESTION: "+ query
    context = "\n EXCERPTS: "+ "\n".join(retrieved_documents)
    if not only_response:
      print("------- retreived documents -------\n")
      for i, doc in enumerate(retrieved_documents):
        print(f"Document {i+1}:")
        print(f"\tDocument Text: {doc}")
      print("------- RAG answer -------\n")
    output = generate_LLM_answer(prompt, context, RAG_LLM)

    display(to_markdown(output))
    print('\n')
    return output

## 2 Test the RAG pipeline

In [ ]:
queries =["Who are the authors suggested a new attention mechanism?",
          "Who are the authors suggested a new controllable text generation mechanism?",
          "Who is Murat Karakaya?",
          "Why do we need to control how the text is produced? ",
          "How can we use the self attention mechanism to control the text generation?",
          "Summarize the paper named Controllable Text Generation",
          "How many blocks are suggested in the transformer?",
          "What about decoder?"
    ]

In [ ]:
reply=generateAnswer(RAG_LLM, chroma_collection, queries[2],10, only_response=False)

------- retreived documents -------

Document 1:
	Document Text: 78 M URAT KARAKAYA TSACO MWSF N μσ μ σ Impro vement ON 900 93. 23 14. 64 109. 33 12. 4 15 % 625 32. 13 8. 50 47. 83 10. 41 33 % 400 6. 76 2. 48 11. 23 3. 86 40 % 225 2. 43 1. 25 3. 03 1. 69 20 % 100 0. 96 0. 88 1. 06 0. 98 9 % TABLE 3Average Number ( μ ) of [UNK] caused by TSACO and MWSF heuristics for different num - be
Document 2:
	Document Text: ediyor. İlgi alanları arasında yapay zeka ve derin öğrenme konuları bulunuyor. Yazılım mühendisi olarak özel bir şirkette çalışmaya devam etmektedir. B. Murat Karakaya KHO Elektrik - Elektronik Mühendisliği nden lisans, Bilkent Üniversitesinden Bilgisayar Mühendisliğinden Yüksek Lisans ve Doktora derecelerini sırasıyla 1991, 2000 ve 2008 yılında almıştır. Şu anda A tılım Üniversitesi Bilgisayar Mühendisliği bölümünde Doç. Dr
Document 3:
	Document Text: Second, the considered chainedcourses are limited to three mathematical courses ( Calculus I, Calculus II, and [UNK] Equa - tio

> Murat Karakaya is an Assistant Professor in the Department of Computer Engineering at Atilim University in Ankara, Turkey.  He is a member of the IACSIT and his research interests include natural computing, sensor networks, peer-to-peer networks, natural computing, optimization, and communications protocol design. 
> 
> This information is found in the excerpt: "He joined the faculty of Atilim University in 2012 and is currently an Asst. Professor in the department of Computer Engineering, Ankara, Turkey. His research interests are natural computi ng, sensor networks, peer - to - peer networks, natural computing, optimization, and communications protocol design. He is a member of IACSIT." 


## 3 A simple loop for the User Interaction

In [ ]:
summarize_collection(chroma_collection)
RAG_LLM.history.clear()
while True:
  question = input("Please enter your question, or type 'bye' to exit: ")
  if question == "bye":
    print("Thank you for using the service. Goodbye!")
    break
  else:
    generateAnswer(RAG_LLM, chroma_collection, question)




Summarizing the collection...
	 Collection name: Papers
	 Number of document chunks in collection: 331
	 Distinct 'document' metadata in the collection:
	  30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf
	  77 Arac Park Yerlerinin Doluluk.pdf
	  15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
	  24 Attention Is All You Need.pdf
	  29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf
	  73 Analyzing Students Academic Success.pdf
Collection summarization completed.
Please enter your question, or type 'bye' to exit: Who are the authors suggested a new attention mechanism?


> I don't know.
> 
> The provided excerpt does not mention who suggested a new attention mechanism. It lists several authors and their works, but there's no specific mention of an attention mechanism being proposed by any of them. 




Please enter your question, or type 'bye' to exit: what is the name of the attention layer?


> I don't know.
> 
> The text excerpt provides information about different attention layers used in the encoder-decoder architecture and the self-attention layer, but it doesn't specifically mention a name for any of these layers. 




Please enter your question, or type 'bye' to exit: Summarize the paper named Attention Is All You Need


> I don't know.
> 
> The text excerpt is not from the paper "Attention Is All You Need," but rather a paper discussing the impact of prerequisite courses on student success in mathematics. It does not contain any information about the paper "Attention Is All You Need." 




Please enter your question, or type 'bye' to exit: how the Ant Colony Optimization is used in the paper?


> The text excerpt describes how Ant Colony Optimization (ACO) is used in the context of **Mobile Sink Scheduling** in Wireless Sensor Networks (WSNs). Here's how:
> 
> 1. **Inspiration from Real Ants:** The ACO algorithm is inspired by how real ants search for food, leaving pheromone trails to guide others.
> 2. **Simulating Ant Behavior:** The paper simulates this behavior by having "ants" traverse the network, representing different possible schedules for the mobile sink (a device that collects data from sensors).
> 3. **Pheromone Deposition:**  As ants explore, they deposit pheromone on edges between visited sensors based on the quality of the schedule they produce.
> 4. **Positive Feedback Loop:** This pheromone deposition creates a positive feedback loop: Successful paths (schedules) attract more ants, increasing pheromone levels, making them more likely to be chosen in future iterations.
> 5. **Convergence to Optimal Solution:** Through this process, the algorithm aims to converge to a near-optimal schedule where the mobile sink visits the most important sensors efficiently.
> 
> The excerpt does not go into the specifics of how the ACO algorithm is implemented within the mobile sink scheduling problem, but it highlights its core concepts. 




Please enter your question, or type 'bye' to exit: bye
Thank you for using the service. Goodbye!


## 4 A Gradio Interface

In [ ]:
%pip install gradio
import gradio as gr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=c3810389b4aec1dc0072dd0e1afa2aa66cd26d35c902aa4a8dfa69937464dbee
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2

In [ ]:
RAG_LLM.history.clear()

# Replace with your actual function (assuming it generates an answer)
def generateAnswerInterFace(question):
    return generateAnswer(RAG_LLM, chroma_collection, question)

# Function to generate the info text
def get_info_text():
    return "INFO: " + summarize_collection(chroma_collection)
    # Assuming summarize_collection returns a string

# Use gr.Blocks instead of gr.Interface
with gr.Blocks() as demo:
    # Define interface components
    query_txt = gr.Textbox(label="Enter your question here:", placeholder="Type your question")
    answer_txt = gr.Textbox(label="Answer:", placeholder="Answer will be displayed here")

    # Create a button to trigger the prediction
    btn = gr.Button("Generate Answer")

    # Define the prediction function (order changed for button placement)
    def predict(question):
        answer = generateAnswerInterFace(question)
        return answer

    info_txt = gr.Textbox(get_info_text(), label="Info")  # Add info textbox after button

    # Connect button click to prediction function
    btn.click(predict, inputs=query_txt, outputs=answer_txt)

# Launch the interface
demo.launch(debug=True)


Summarizing the collection...
	 Collection name: Papers
	 Number of document chunks in collection: 331
	 Distinct 'document' metadata in the collection:
	  30 Time-Sensitive Ant Colony Optimization ORIGINAL.pdf
	  77 Arac Park Yerlerinin Doluluk.pdf
	  15 UAV Route Planning For Maximum Target Coverage ABSTRACT.pdf
	  24 Attention Is All You Need.pdf
	  29 Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty ABSTRACT.pdf
	  73 Analyzing Students Academic Success.pdf
Collection summarization completed.
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://93a05a81c69ca8683d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio dep

> I don't know.
> 
> The text provided does not mention who Murat is. It does mention Murat KARAKAYA, but it does not give any information about who he is. 


> Murat Karakaya is an Assistant Professor in the Department of Computer Engineering at Atilim University in Ankara, Turkey. 
> 
> Here's how we know this:
> 
> 1. The text states: "B. Murat Karakaya... currently an Asst. Professor in the department of Computer Engineering, Ankara, Turkey."
> 2. The text also includes his email address: "murat.karakata@atilim.edu.tr" which further confirms his affiliation with Atilim University. 
> 
> The text goes on to describe his research interests and academic background. 


> Attention, in this context, is a mechanism that allows a model to focus on specific parts of an input sequence when processing it. 
> 
> Here's a breakdown based on the provided text:
> 
> 1. **Self-attention (intra-attention):**  This type of attention relates different positions within a single sequence (e.g., a sentence) to understand its structure and meaning better.
> 2. **Multi-Head Attention:** This is a more advanced approach where multiple attention layers run in parallel, allowing the model to focus on different aspects of the input simultaneously.
> 3. **Scaled Dot-Product Attention:** This is a specific type of attention where the model calculates a score based on the similarity between a "query" and "key" to determine the weight assigned to each value.
> 
> The text discusses how attention mechanisms are used in language models to learn long-range dependencies, which helps them understand the relationships between words and phrases in a sentence. Examples of this are shown in the figures, where different attention heads focus on different aspects of the input. 


> This chat session started with you asking about who "Murat" is. The provided text did not have information about a person named "Murat" but it did mention "Murat Karakaya".  The second question was about who "Murat Karakaya" is and the answer was that he is an Assistant Professor in the Department of Computer Engineering at Atilim University in Ankara, Turkey.  The final question was about what "attention" is, which is a mechanism that allows a model to focus on specific parts of an input sequence when processing it. 




Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://93a05a81c69ca8683d.gradio.live


In [ ]:
RAG_LLM.history

[parts {
   text: "QUESTION: who is Murat\n EXCERPTS: ediyor. \304\260lgi alanlar\304\261 aras\304\261nda yapay zeka ve derin \303\266\304\237renme konular\304\261 bulunuyor. Yaz\304\261l\304\261m m\303\274hendisi olarak \303\266zel bir \305\237irkette \303\247al\304\261\305\237maya devam etmektedir. B. Murat Karakaya KHO Elektrik - Elektronik M\303\274hendisli\304\237i nden lisans, Bilkent \303\234niversitesinden Bilgisayar M\303\274hendisli\304\237inden Y\303\274ksek Lisans ve Doktora derecelerini s\304\261ras\304\261yla 1991, 2000 ve 2008 y\304\261l\304\261nda alm\304\261\305\237t\304\261r. \305\236u anda A t\304\261l\304\261m \303\234niversitesi Bilgisayar M\303\274hendisli\304\237i b\303\266l\303\274m\303\274nde Do\303\247. Dr\n. edu. tr ). Mobile Sink Scheduling Method for Wireless Sensor Networks under Travel Time Uncertainty Murat KARAKAYA, Member, SAISE Author [UNK] s formal photo\nwhile posing their own pheromone on the way as well. As a result, the more com - monly used path

# SUMMARY

WHY WE NEED A PERSISTENT CHROMADB?
In the context of a Retrieval-Augmented Generation (RAG) approach, saving and loading a persistent ChromaDB is particularly important for several reasons:

Enhanced Data Durability:

Importance: Ensures the retrieval database used for augmenting generative models is not lost between sessions or system restarts.
RAG Relevance: Maintains a consistent and reliable knowledge base that the generative model can reference, leading to more accurate and relevant responses.
Operational Continuity:

Importance: Allows seamless continuation of operations without needing to re-index or re-import data, saving time and computational resources.
RAG Relevance: Ensures that the generative model has continuous access to the same set of documents, which is essential for generating consistent and coherent responses over time.
Facilitating Collaboration:

Importance: Enables multiple users or systems to share and access the same dataset.
RAG Relevance: Supports collaborative development and usage of the RAG system, allowing different teams to work on improving the retrieval and generation processes simultaneously.
Scalability:

Importance: Provides a stable and persistent backend, enabling efficient handling of large datasets.
RAG Relevance: Essential for scaling the RAG system to handle more extensive and diverse knowledge bases, ensuring that the system can manage increased loads and deliver prompt, relevant information.
In a RAG system, the retriever (like ChromaDB) provides the generative model with relevant context from a knowledge base to generate informed and accurate responses. Persistent storage ensures that this knowledge base is durable, continuously available, and scalable, which is critical for the reliability, consistency, and performance of the RAG system.

.

.

.

In [ ]:
def generateAnswer(RAG_LLM, chroma_collection,query,n_results=5):
    retrieved_documents= retrieveDocs(chroma_collection, query, 10, return_only_docs=True)

    print("------- retreived documents -------\n")
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print(f"\tDocument Text: {doc}")
    prompt = "QUESTION: "+ query
    context = "\n EXCERPTS: "+ "\n".join(retrieved_documents)
    print("------- RAG answer -------\n")
    output = generate_LLM_answer(prompt, context, RAG_LLM)

    display(to_markdown(output))
    print('\n')
    return output

In [ ]:
to_markdown(reply.text)

AttributeError: 'str' object has no attribute 'text'

In [ ]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))


In [ ]:
model.count_tokens(chat.history)

In [ ]:
response = chat.send_message(prompt)
to_markdown(response.text)


In [ ]:
import os
import openai
from openai import OpenAI

'''
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
openai_client = OpenAI()
'''
openai_client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo-1106"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "As an attentive and supportive academic assistant, "
            "your task is to provide assistance based solely on the provided"
            " excerpts. Answer the following questions, ensuring your responses"
            " are derived exclusively from the provided partial texts. "
            "If the answer cannot be found within the provided excerpts, "
            "kindly respond with 'I don't know'."
            "After answering each question, please provide a detailed "
            "explanation, breaking down the answer step by step and relating "
            "it to the provided excerpts."
            "Return your response as a Json object with two key fields: "
            " 'Answer', which should contain the value of the answer, and "
            " 'Reason', which should provide an explanation of why this answer "
            "was generated."

        },
        {"role": "user", "content": f"Question: {query}. \n Excerpts: {information}"}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
def generateAnswer(query,n_results=5):
    retrieved_documents=retrieveDocs(query,n_results)

    print("------- retreived documents -------\n")
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print(f"\tDocument Text: {doc}")
    print("------- RAG answer -------\n")
    output = rag(query=query, retrieved_documents=retrieved_documents)
    print(output)
    print('\n')
    return output

In [ ]:
reply=generateAnswer(queries[5],10)


In [ ]:
# prompt: convert the 'reply' to a dict

import ast
reply_dict = ast.literal_eval(reply)
print(f"Answer: {reply_dict['Answer']}")
print(f"Because; {reply_dict['Reason']}")

In [ ]:
for query in queries:
  generateAnswer(query)

In [ ]:
%pip install umap-learn

In [ ]:
def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings),2))
    for i, embedding in enumerate(tqdm(embeddings)):
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings

In [ ]:
import umap.umap_ as umap

embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10)
plt.gca().set_aspect('equal', 'datalim')
plt.title('Projected Embeddings')
plt.axis('off')

In [ ]:
query = queries[3]

results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(document)
    print('')


In [ ]:
query_embedding = embedding_function([query])[0]
retrieved_embeddings = results['embeddings'][0]

projected_query_embedding = project_embeddings([query_embedding], umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)


In [ ]:
# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_query_embedding[:, 0], projected_query_embedding[:, 1], s=150, marker='X', color='r')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none', edgecolors='g')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{query}')
plt.axis('off')

In [ ]:
def augment_query_generated(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "Sen TÜBİTAK proje başvurularını inceleyen yapay zeka konusunda uzman bir akasemisyensin."
            "Aşağıda verilen soruya, aşağıdaki proje tanımına uygun olabilecek bir cevap üret: \n"
            "Projenin genel amacı, bankacılık sektöründeki risk yönetimi operasyonlarını geliştirmek ve finansal kurumların karşılaştığı zorlukları ele almak "
            "için yapay zeka (AI) tabanlı bir platform geliştirmektir. Proje, bankalara vadeli mevduatın erken bozulması, kredilerin erken ödenmesi ve çeşitli "
            "mevduat türlerinin belirlenmesi gibi davranışsal riskleri daha etkili bir şekilde yönetme kapasitesi sunmayı hedeflemektedir. Bu riskler, finansal "
            "kurumların bilanço dengesini etkileyebilir ve operasyonel verimliliği azaltabilir. "
            "Projenin çözmeyi amaçladığı temel problem, bankaların karlılık ve risk analizlerini gerçekleştirirken karşılaştığı karmaşık durumları doğru ve "
            "etkili bir şekilde yönetme ihtiyacıdır. Özellikle vadeli mevduatların erken kapanması ve kredilerin erken ödenmesi gibi durumlar, bankaların "
            "gelecekteki nakit akışlarını ve risk profillerini belirleme sürecini karmaşıklaştırabilir"

        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
original_query = queries[0]
hypothetical_answer = augment_query_generated(original_query)

joint_query = f"{original_query} {hypothetical_answer}"
print(joint_query)

In [ ]:
def extend_retrieved_documents(results, extension=4):
  original_ids= results['ids'][0]
  print("original_ids: ",original_ids)

  extended_ids = set()


  for id in original_ids:
    extended_ids.add(int(id))
    for i in range(1, extension):
      extended_ids.add(int(id)+i)


  extended_ids = sorted([int(x) for x in extended_ids])
  extended_ids = [str(x) for x in extended_ids if int(x) < chroma_collection.count()]
  print("extended_ids: ",extended_ids)
  return chroma_collection.get(extended_ids)['documents']

In [ ]:
def retrieveDocs_augmented_query(query, n_results=5, extension=4):
    hypothetical_answer = augment_query_generated(query)
    print("------ hypothetical_answer ---------\n")
    print(hypothetical_answer,"\n")
    print("------------------------------------\n")
    joint_query = f"{query} {hypothetical_answer}"
    results = chroma_collection.query(query_texts=joint_query, n_results=n_results, include=['documents', 'embeddings'])
    retrieved_documents = extend_retrieved_documents(results, extension)
    #retrieved_documents = results['documents'][0]

    return retrieved_documents



In [ ]:
retrieved_documents=retrieveDocs_augmented_query(query, 5)

for doc in retrieved_documents:
    print(doc)
    print('')

In [ ]:
results = chroma_collection.query(query_texts=joint_query, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]

for doc in retrieved_documents:
    print(doc)
    print('')

In [ ]:
retrieved_embeddings = results['embeddings'][0]
original_query_embedding = embedding_function([original_query])
augmented_query_embedding = embedding_function([joint_query])

projected_original_query_embedding = project_embeddings(original_query_embedding, umap_transform)
projected_augmented_query_embedding = project_embeddings(augmented_query_embedding, umap_transform)
projected_retrieved_embeddings = project_embeddings(retrieved_embeddings, umap_transform)

In [ ]:
import matplotlib.pyplot as plt

# Plot the projected query and retrieved documents in the embedding space
plt.figure()
plt.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10, color='gray')
plt.scatter(projected_retrieved_embeddings[:, 0], projected_retrieved_embeddings[:, 1], s=100, facecolors='none', edgecolors='g')
plt.scatter(projected_original_query_embedding[:, 0], projected_original_query_embedding[:, 1], s=150, marker='X', color='r')
plt.scatter(projected_augmented_query_embedding[:, 0], projected_augmented_query_embedding[:, 1], s=150, marker='X', color='orange')

plt.gca().set_aspect('equal', 'datalim')
plt.title(f'{original_query}')
plt.axis('off')

In [ ]:
def generateAnswer_augmented_query(query,n_results=5, extention=4):
    print("------- query -------\n")
    print(query,"\n")
    retrieved_documents=retrieveDocs_augmented_query(query,n_results,extention)
    print("------- retreived documents -------\n")
    for document in retrieved_documents:
        print(document)
        print('\n')

    print("------- RAG answer -------\n")
    output = rag(query=query, retrieved_documents=retrieved_documents)
    print(output)
    print('\n')

In [ ]:
queries

In [ ]:
generateAnswer_augmented_query(queries[0],10,5)

In [ ]:
title= """Ar-Ge Sürecinde Kullanılacak Yöntemler Tanımlanan proje hedeflerine ulaşmak için uygulanacak analitik
        deneysel çözüm yöntemlerini belirtiniz. (NOT: Bu bölümde sunulan proje özelinde
        hangi teknik / bilimsel yaklaşımların ve bunlara ait aşamaların takip edileceği açıklanmalı, iş paketleri isimleri ya da her projede olabilecek standart
        rutin çalışma yöntemleri tekrarlanmamalıdır."""
results = chroma_collection.query(query_texts=title, n_results=5, include=['documents', 'embeddings'])
retrieved_documents = results['documents'][0]
print(retrieved_documents)

In [ ]:
title= """Ar-Ge Sürecinde Kullanılacak Yöntemler Tanımlanan proje hedeflerine ulaşmak için uygulanacak analitik
        deneysel çözüm yöntemlerini belirtiniz. (NOT: Bu bölümde sunulan proje özelinde
        hangi teknik / bilimsel yaklaşımların ve bunlara ait aşamaların takip edileceği açıklanmalı, iş paketleri isimleri ya da her projede olabilecek standart
        rutin çalışma yöntemleri tekrarlanmamalıdır."""
results = chroma_collection.query(query_texts=title, n_results=5, include=['documents', 'embeddings'])

retrieved_documents = extend_retrieved_documents(results)
print(retrieved_documents)


In [ ]:
chroma_collection.get(results['ids'][0])